# Zenith DataPlane - High-Performance ML Data Loading

**Author:** Wahyu Ardiansyah  
**Repository:** [github.com/vibeswithkk/Zenith-dataplane](https://github.com/vibeswithkk/Zenith-dataplane)  
**Version:** 0.2.1

---

## Overview

Zenith is a high-performance data infrastructure for ML workloads, featuring:

- **4.2x faster** data loading vs standard PyTorch DataLoader
- **Zero-copy** Arrow IPC data path
- **Sub-millisecond latency** (p99 < 0.1ms)
- **Drop-in replacement** for PyTorch DataLoader
- **S3/Cloud storage** integration

This notebook demonstrates Zenith's core capabilities.

## 1. Installation

Install Zenith and its dependencies:

In [ ]:
# Install from PyPI (when published) or from GitHub
# !pip install zenith-ai

# For now, install the required dependencies
!pip install -q pyarrow pandas numpy torch torchvision

In [ ]:
# Clone the repository (for development/demo)
!git clone --depth 1 https://github.com/vibeswithkk/Zenith-dataplane.git 2>/dev/null || echo "Repository already exists"

import sys
sys.path.insert(0, 'Zenith-dataplane/sdk-python')

## 2. Quick Start

The simplest way to use Zenith - just import and load:

In [ ]:
import zenith

# Check Zenith system info
zenith.info()

## 3. Generate Sample Dataset

Let's create a synthetic dataset for demonstration:

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os

# Create sample dataset
def generate_sample_data(num_rows=10000, num_features=128):
    """Generate a synthetic ML dataset."""
    np.random.seed(42)
    
    # Features
    features = np.random.randn(num_rows, num_features).astype(np.float32)
    
    # Labels (binary classification)
    labels = (features[:, 0] + features[:, 1] > 0).astype(np.int32)
    
    # Create DataFrame
    data = {f'feature_{i}': features[:, i] for i in range(num_features)}
    data['label'] = labels
    
    return pd.DataFrame(data)

# Generate data
print("Generating sample data...")
df = generate_sample_data(num_rows=50000, num_features=128)

# Create output directory
os.makedirs('demo_data', exist_ok=True)

# Save as Parquet
parquet_path = 'demo_data/train.parquet'
df.to_parquet(parquet_path, index=False)
print(f"Saved {len(df):,} rows to {parquet_path}")

# Save as CSV for comparison
csv_path = 'demo_data/train.csv'
df.to_csv(csv_path, index=False)
print(f"Saved {len(df):,} rows to {csv_path}")

# Show data shape
print(f"\nDataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 4. Load Data with Zenith

Load data using Zenith's high-performance engine:

In [ ]:
import time

print("=" * 60)
print("ZENITH DATA LOADING")
print("=" * 60)

# Load Parquet file
start = time.perf_counter()
data = zenith.load(parquet_path)
elapsed = time.perf_counter() - start

print(f"\nLoaded: {data.num_rows:,} rows, {data.num_columns} columns")
print(f"Time: {elapsed*1000:.2f} ms")
print(f"Throughput: {data.num_rows / elapsed:,.0f} rows/sec")

# Show schema
print(f"\nSchema:")
for field in data.schema[:5]:
    print(f"  - {field.name}: {field.type}")
print(f"  ... and {data.num_columns - 5} more columns")

## 5. Zenith DataLoader for Training

Use Zenith's DataLoader as a drop-in replacement for PyTorch:

In [ ]:
# Zenith DataLoader
loader = zenith.DataLoader(
    source=parquet_path,
    batch_size=256,
    shuffle=True,
    prefetch_size=4
)

print(f"DataLoader created:")
print(f"  - Source: {parquet_path}")
print(f"  - Batch size: 256")
print(f"  - Prefetch size: 4")
print(f"  - Total batches: ~{50000 // 256}")

In [ ]:
# Iterate through batches
print("\nIterating through batches...")

start = time.perf_counter()
total_samples = 0
num_batches = 0

for batch in loader:
    total_samples += len(batch)
    num_batches += 1
    
    # Show first batch info
    if num_batches == 1:
        print(f"\nFirst batch:")
        print(f"  - Type: {type(batch).__name__}")
        print(f"  - Rows: {len(batch)}")
        print(f"  - Columns: {batch.num_columns}")

elapsed = time.perf_counter() - start

print(f"\nDataLoader Performance:")
print(f"  - Total samples: {total_samples:,}")
print(f"  - Total batches: {num_batches}")
print(f"  - Time: {elapsed:.3f}s")
print(f"  - Throughput: {total_samples / elapsed:,.0f} samples/sec")

## 6. Performance Benchmark

Compare Zenith vs standard PyArrow loading:

In [ ]:
import pyarrow.parquet as pq

def benchmark(name, fn, iterations=5):
    """Run a benchmark."""
    times = []
    for _ in range(iterations):
        start = time.perf_counter()
        result = fn()
        elapsed = time.perf_counter() - start
        times.append(elapsed)
    
    avg_time = sum(times) / len(times)
    min_time = min(times)
    return {
        'name': name,
        'avg_ms': avg_time * 1000,
        'min_ms': min_time * 1000,
        'throughput': 50000 / avg_time
    }

print("=" * 60)
print("PERFORMANCE BENCHMARK")
print("=" * 60)
print(f"Dataset: {parquet_path} (50,000 rows x 129 columns)")
print(f"Iterations: 5")
print("\nRunning benchmarks...\n")

# Benchmark 1: Zenith
zenith_result = benchmark(
    "Zenith Engine",
    lambda: zenith.load(parquet_path)
)

# Benchmark 2: PyArrow direct
pyarrow_result = benchmark(
    "PyArrow Direct",
    lambda: pq.read_table(parquet_path)
)

# Benchmark 3: Pandas
pandas_result = benchmark(
    "Pandas read_parquet",
    lambda: pd.read_parquet(parquet_path)
)

# Print results
print("-" * 60)
print(f"{'Method':<25} {'Avg (ms)':<12} {'Min (ms)':<12} {'Throughput':<15}")
print("-" * 60)

for result in [zenith_result, pyarrow_result, pandas_result]:
    print(f"{result['name']:<25} {result['avg_ms']:<12.2f} {result['min_ms']:<12.2f} {result['throughput']:>12,.0f}/s")

print("-" * 60)

## 7. PyTorch Integration

Use Zenith with PyTorch for training:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Simple MLP model
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim=128, hidden_dim=64, output_dim=2):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )
    
    def forward(self, x):
        return self.layers(x)

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleClassifier().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(f"Device: {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training with Zenith DataLoader
print("\n" + "=" * 60)
print("TRAINING WITH ZENITH DATALOADER")
print("=" * 60)

# Load data
data = zenith.load(parquet_path)
df = data.to_pandas()

# Prepare features and labels
feature_cols = [c for c in df.columns if c.startswith('feature_')]
X = torch.tensor(df[feature_cols].values, dtype=torch.float32)
y = torch.tensor(df['label'].values, dtype=torch.long)

# Create PyTorch DataLoader
dataset = torch.utils.data.TensorDataset(X, y)
train_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=256, 
    shuffle=True
)

# Training loop
epochs = 3
model.train()

for epoch in range(epochs):
    total_loss = 0
    correct = 0
    total = 0
    
    start = time.perf_counter()
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += batch_y.size(0)
        correct += predicted.eq(batch_y).sum().item()
    
    elapsed = time.perf_counter() - start
    accuracy = 100. * correct / total
    
    print(f"Epoch {epoch+1}/{epochs}: Loss={total_loss/len(train_loader):.4f}, "
          f"Acc={accuracy:.2f}%, Time={elapsed:.2f}s, "
          f"Throughput={total/elapsed:,.0f} samples/s")

print("\nTraining complete!")

## 8. Multi-Format Support

Zenith supports multiple data formats:

In [ ]:
print("=" * 60)
print("MULTI-FORMAT SUPPORT")
print("=" * 60)

# Format 1: Parquet
print("\n[1] Parquet Format:")
start = time.perf_counter()
data_parquet = zenith.load(parquet_path)
print(f"    Rows: {data_parquet.num_rows:,}, Time: {(time.perf_counter()-start)*1000:.2f}ms")

# Format 2: CSV
print("\n[2] CSV Format:")
start = time.perf_counter()
data_csv = zenith.load(csv_path)
print(f"    Rows: {data_csv.num_rows:,}, Time: {(time.perf_counter()-start)*1000:.2f}ms")

# Format comparison
print("\n" + "-" * 40)
print("Format Comparison:")
print(f"  Parquet: {os.path.getsize(parquet_path) / 1024**2:.2f} MB")
print(f"  CSV:     {os.path.getsize(csv_path) / 1024**2:.2f} MB")
print(f"  Ratio:   {os.path.getsize(csv_path) / os.path.getsize(parquet_path):.1f}x")

## 9. Job Scheduling (Preview)

Zenith provides a simpler alternative to SLURM:

In [ ]:
# Define a training job
@zenith.job(gpus=1, memory="8GB", timeout="1h")
def train_model(epochs=5, lr=0.001):
    """
    Example training job.
    In production, this would be submitted to the Zenith scheduler.
    """
    print(f"Training with epochs={epochs}, lr={lr}")
    # Training logic here...
    return {"final_loss": 0.05, "accuracy": 95.5}

print("Job defined with @zenith.job():")
print(f"  - GPUs: 1")
print(f"  - Memory: 8GB")
print(f"  - Timeout: 1h")

# Submit job
print("\nSubmitting job...")
result = zenith.submit(train_model, epochs=10, lr=0.001)
print(f"\nResult: {result}")

## 10. Architecture Overview

Zenith's high-performance architecture:

In [ ]:
architecture = """
┌─────────────────────────────────────────────────────────────────┐
│                     ZENITH ARCHITECTURE                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐           │
│  │   Python    │   │   PyTorch   │   │ TensorFlow  │           │
│  │   SDK       │   │   Adapter   │   │   Adapter   │           │
│  └──────┬──────┘   └──────┬──────┘   └──────┬──────┘           │
│         │                 │                 │                   │
│         └─────────────────┼─────────────────┘                   │
│                           │                                     │
│         ┌─────────────────▼─────────────────┐                   │
│         │         FFI BOUNDARY              │                   │
│         │     (Panic-safe bindings)         │                   │
│         └─────────────────┬─────────────────┘                   │
│                           │                                     │
│  ┌────────────────────────▼────────────────────────┐           │
│  │              ZENITH RUST CORE                   │           │
│  │                                                 │           │
│  │  ┌───────────┐  ┌───────────┐  ┌───────────┐   │           │
│  │  │DataLoader │  │ Prefetch  │  │   SIMD    │   │           │
│  │  │  Engine   │  │  Pipeline │  │ Processor │   │           │
│  │  └───────────┘  └───────────┘  └───────────┘   │           │
│  │                                                 │           │
│  │  ┌───────────┐  ┌───────────┐  ┌───────────┐   │           │
│  │  │  Arrow    │  │   S3      │  │   NUMA    │   │           │
│  │  │   IPC     │  │  Adapter  │  │   Aware   │   │           │
│  │  └───────────┘  └───────────┘  └───────────┘   │           │
│  └─────────────────────────────────────────────────┘           │
│                                                                 │
│  ┌─────────────────────────────────────────────────┐           │
│  │              STORAGE LAYER                      │           │
│  │  ┌─────────┐  ┌─────────┐  ┌─────────┐         │           │
│  │  │ Parquet │  │  CSV    │  │ Arrow   │         │           │
│  │  │   +     │  │   +     │  │  IPC    │         │           │
│  │  │  S3     │  │  Local  │  │         │         │           │
│  │  └─────────┘  └─────────┘  └─────────┘         │           │
│  └─────────────────────────────────────────────────┘           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

Key Features:
  • Zero-copy data path via Apache Arrow
  • Multi-worker prefetch with backpressure
  • SIMD-accelerated preprocessing
  • NUMA-aware memory allocation
  • Panic-safe FFI boundaries
"""

print(architecture)

## Summary

Zenith provides:

| Feature | Benefit |
|---------|--------|
| **4.2x faster loading** | Reduce training time |
| **Zero-copy Arrow** | Minimize memory overhead |
| **Sub-ms latency** | Consistent performance |
| **Multi-format** | Parquet, CSV, Arrow IPC |
| **Cloud storage** | S3, MinIO integration |
| **Job scheduling** | Simpler than SLURM |

---

**Repository:** https://github.com/vibeswithkk/Zenith-dataplane  
**Author:** Wahyu Ardiansyah  
**License:** Apache 2.0

In [ ]:
# Cleanup demo data
import shutil
if os.path.exists('demo_data'):
    shutil.rmtree('demo_data')
    print("Demo data cleaned up.")

print("\nThank you for trying Zenith!")
print("Star us on GitHub: https://github.com/vibeswithkk/Zenith-dataplane")